<a href="https://colab.research.google.com/github/AlejandroMllo/Machine-Learning-Algorithms/blob/master/Evolutionary%20Algorithms/StrengthParetoEvolutionaryAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Strength Pareto Evolutionary Algorithm (SPEA & SPEA2)

## SPEA *(Zitler & Thiele, 1998)*

Similar to other multiobjective EAs it:
- stores the Pareto-optimal solutions found so far externally,
- uses the concept of Pareto dominance in order to assign scalar fitness values to individuals, and
- performs clustering to reduce the number of nondominated solutions stored without destroying the characteristics of the Pareto-optimal front.

On the other hand, SPEA is unique in four respects:
- It combines the above three techniques in a single algorithm.
- The fitness of an individual is determined from the solutions stored in the external Pareto set only; whether members of the Population dominate each other is irrelevant.
- All solutions in the external Pareto set participate in selection.
- A new niching method is provided in order to preserve diversity in the Population; this method is Pareto-based and does no requiere any distance parameter (like the niche radius for sharing).

### Outline:

On iteration consists of:
1. The external Pareto Set (Archive) is updated: all nondominated individuals in the Population are copied to the Pareto set and, subsequently, possibly dominated solutions are removed from it.
2. If the number of externally stored Pareto solutions exceeds a given maximum, a reduced representation is computed by clustering.
3. After Fitness assignment, individuals randomly picked out of the union of Population and Pareto set hold binary tournaments in order to fill the Mating pool.
4. Finally, Crossover and Mutation are applied to the Population as usual, getting the Generation $i + 1$ and starting again up to Generation $n$.

#### Fitness asignment procedure:
In a first step each solution in the Pareto set is assigned a (real) value $s \in [0, 1)$, called $strength$; $s$ is proportional to the number of Population members covered. The strength of a Pareto solution is at the same time its Fitness. Subsequently, in the second step, the individuals in the Population are ranked in the aforementioned manner, though with respect to the calculated strengths. That is, for each individual the strengths of all external Pareto solutions by which it is covered are summed up. We add one to the resulting value (in order to guarantee that Pareto solutions are most likely to be reproduced) and obtain the Fitness valu $f$, where $f \in [1, N)$ and $N$ denotes the Population size.
If all Pareto solutions have equal strengths, the fitness of an individual is completely determined by the number of Pareto points that cover it. However, in the case the Population is unbalanced the strengths come into play: the stronger a Pareto point the less fitter the covered individuals.

#### Pareto Set reduction:
The goal is not only to prune a given set, but rather to generate a representative subset which maintains the characteristics of the original set. Cluster analysis fits these requirements.\
In general, Cluster Analysis partitions a collection of $p$ elements into $q$ groups of relatively homogeneous elements, where $q < p$. Dependent on the working mechanism of the algorithm two forms of clustering are distinguished: direct clustering and hierarchical clustering. While the first approach sorts the $p$ elements into $q$ groups in one step, the latter works iteratively by joining adjacent clusters until the required number of groups is obtained.\
\
SPEA uses a hierarchical method, which is _Average linkage method_.\
At the beginning each element of the original Pareto set forms a basic cluster. Then, in each step two clusters are chosen to amalgamate into a larger cluster until the given number of clusters is reached. The two clusters are selected according to the nearest neighbor criterion, where the distance between two clusters is given as the average distance between pairs of individuals across clusters. Finally, when the partitioning procedure is finished, the reduced Pareto Set is formed by selecting a representative individual for each cluster. We consider the centroid (the point with minimal average distance to all other points in the cluster) as representative solution.


---

## SPEA2 *(Zitler et al., 2001)*

SPEA forms the basis for SPEA2.

### Outline:
1. Initialization: generate an initial Population $P_0$ and create the empty Archive (external set / Pareto Set) $\bar{P_0} = \emptyset$. Set $t = 0$.
2. Fitness assignment: calculate fitness values of individuals in $P_t$ and $\bar{P_t}$.
3. Environmental selection: copy all nondominated individuals in $P_t$ and $\bar{P_t}$ to $\bar{P}_{t+1}$. If size of  $\bar{P}_{t+1}$ exceeds the Archive size ($\bar{N}$) then reduce  $\bar{P}_{t+1}$ by means of the truncation operator, otherwise if size of  $\bar{P}_{t+1}$ is less than $\bar{N}$ then fill  $\bar{P}_{t+1}$ with dominated individuals in  $P_t$ and  $\bar{P}_{t}$
4. Termination: if $t \geq T$,  where $T$ is the maximum number of generations, or another stopping criterion is satisfied the set $A$ to the set of decision vectors represented by the nondominated individuals in  $\bar{P}_{t+1}$. Stop.
5. Mating selection: perform binary tournament selection with replacement on  $\bar{P}_{t+1}$ in order to fill the mating pool.
6. Variation: apply Recombination and Mutation operators to the mating pool and set  $P_{t+1}$ to the resulting Population. Increment generation counter $(t = t + 1)$ and go to Step 2.

#### Fitness assignment procedure:
To avoid the situation that individuals dominated by the same Archive members have identical fitness values, with SPEA2 for each individual both dominating and dominated solutions are taken into account.\
In detail, each individual $i$ in the archive $\bar{P_t}$ and the Population $P_t$ is assigned a strength value $S(i)$, representing the number of solutions it dominates:
$$ S(i) = |\{j\ |\ j \in P_t + \bar{P_t} \wedge i \succ j\}| $$
where $| \cdot |$ denotes the cardinality of a set, $+$ stands for multiset union and the symbol $\succ$ corresponds to the Pareto dominance relation. On the basis of the _S_ values, the raw fitness $R(i)$ of an individual $i$ is calculated:
$$ R(i) = \Sigma_{j \in P_t + \bar{P_t},\ j \succ i} S(j) $$
That is, the raw fitness is determined by the strengths of its dominators in both Archive and Population. Fitness is to be minimized here.\
Additional density information is incorporated to discriminate between individuals having identical raw fitness values. The density estimation technique used is an adaptation of the _k_-nearest-neighbor method, where the density at any point is a (decreasing) function to the distance to the _k_-th nearest data point. Here, we simply take the inverse of the distance to the _k_-th nearest neighbor as the density estimate. To be more precise, for each individual $i$ the distances (in objective space) to all individuals $j$ in Archive and Population are calculated and stored in a list. After sorting the list in increasing order, the _k_-th element gives the distance sought, denoted as $\sigma_i^k$. As a common setting, we use _k_ equal to the square root of the sample size. Afterwards, the density $D(i)$ corresponding to $i$ is defined by:
$$ D(i) = \frac{1}{\sigma_i^k + 2} $$
In the denominator, two is added to ensure that its value is greater than zero and that $D(i) \lt 1$. \
Finally, adding $D(i)$ to the raw fitness value $R(i)$ of an individual $i$ yields its fitness $F(i)$:
$$ F(i) = R(i) + D(i) $$

#### Environmental selection:
The Archive update operation (Step 3) in SPEA2 differs from the one in SPEA in two respects:
- the number of individuals contained in the Archive is constant over time, and
- the truncation method prevents boundary solutions being removed.\
During environmental selection, the first step is to copy all nondominated individuals, i.e., those which have a fitness lower than one, from Archive and Population to the Archive of the next generation:
$$ \bar{P}_{t+1} = \{\ i\ |\ i \in P_t + \bar{P}_{t} \wedge F(i) \lt 1\} $$
If the nondominated front fits exactly into the Archive $(\bar{P}_{t+1} = \bar{N})$ the environmental selection step is completed. Otherwise, there can be two situations: Either the Archive is too small $(\bar{P}_{t+1} \lt \bar{N})$ or too large $(\bar{P}_{t+1} \gt \bar{N})$. In the first case, the best $\bar{N} - |\bar{P}_{t+1}| $ dominated individuals in the previous Archive and Population are copied to the new Archive. This can be implemented by sorting the multiset $P_t + \bar{P}_t$ according to the fitness values and copy the first $\bar{N} - |\bar{P}_{t+1}| $ individuals $i$ with $ F(i) \geq 1$ from the resulting ordered list to $\bar{P}_{t+1}$. In the second case, when the size of the current nondominated (multi)set exceeds $\bar{N}$, an Archive truncation procedure is invoked which iteratively removes individuals from $\bar{P}_{t+1}$ until $\bar{P}_{t+1} = \bar{N}$. Here, at each iteration that individual $i$ is chosen for removal for which $i \leq_d j\ \forall j \in \bar{P}_{t+1}$ with
$$i \leq_d j\ :\iff \forall\ 0 \lt k \lt |\bar{P}_{t+1}|\ :\ \sigma_i^k = \sigma_j^k \vee \\ \exists\ 0 \lt k \lt |\bar{P}_t|\ :\ [(\forall\ 0 \lt l \lt k\ : \sigma_i^l = \sigma_j^l) \wedge \sigma_i^k = \sigma_j^k] $$
where $\sigma_i^k$ denotes the distance of $i$ to its _k_-th nearest neighbor in $\bar{P}_{t+1}$. In other words, the individual which has the minimum distance to another individual is chosen at each stage; if there are several individuals with minimum distance the tie is broken by considering the second smallest distances and so forth.


### Example using SPEA2

Solution of the multiobjective optimization problem SCH, problem 1 described in (Deb et al., 2002), given by:

$$\\ \min   f_1(x) + f_2(x) $$

$where$
      $$  f_1(x) = \Sigma_1^n (x_i^2) $$
      $$ f_2(x) = \Sigma_1^n (x_i - 2)^2 $$
     
$subject\ to$
    $$ -10 \leq x_i \leq 10, n = 1$$

$whose\ optimal\ value\ is:\ x \in [0, 2].$

using SPEA2, and based on the implementation by Brownlee (Brownlee, 2015).

In [0]:
class Individual:
  
  def __init__(self):
        
    self.__bitstring = None
    self.__vector = None
    
    self.__objectives = None
    
    self.__dominated_set = None
    self.__distance = None
    
    self.__raw_fitness = None
    self.__density = None
    
  # Getters
  def bitstring(self):
    return self.__bitstring
  
  def vector(self):
    return  self.__vector
  
  def objectives(self):
    return self.__objectives
  
  def dominated_set(self):
    return self.__dominated_set
  
  def distance(self):
    return self.__distance
  
  def raw_fitness(self):
    return self.__raw_fitness
  
  def density(self):
    return self.__density
  
  def fitness(self):
    return self.density() + self.raw_fitness()
  
  # Setters
  def set_bitstring(self, bitstring):
    self.__bitstring = bitstring
    
  def set_vector(self, vector):
    self.__vector = vector
    
  def set_objectives(self, objectives):
    self.__objectives = np.array(objectives)
    
  def set_dominated_set(self, dom_set):
    self.__dominated_set = dom_set
    
  def set_distance(self, dist):
    self.__distance = dist
  
  def set_raw_fitness(self, raw_fitness):
    self.__raw_fitness = raw_fitness
    
  def set_density(self, density):
    self.__density = density
  

In [0]:
# Required Libraries
import numpy as np

from random import random, randint

In [0]:
# Objective Functions

def f1(x):
  
  x = x**2
  return np.sum(x)


def f2(x):
  
  x = (x - 2)**2
  return np.sum(x)

In [0]:
# Helper Functions

def decode(bitstring, search_space, bits_per_param):
  """
  _genotype_ to _fenotype_ conversion.
  """
  
  vector = np.array([])
  
  for index, bound in enumerate(search_space):
    
    off, sumation = index*bits_per_param, 0.0
    param = list(reversed(bitstring[off : off + bits_per_param]))
    
    for j in range(len(param)):
      sumation += float((1.0 if param[j] == '1' else 0.0) * (2.0 ** j))
      
    minimum, maximum = bound
    
    new_element = \
      minimum + ( (maximum - minimum)/((2.0**bits_per_param) - 1.0) ) * sumation
    vector = np.append(vector, new_element)
    
  return vector


def random_bitstring(num_bits):
  
  bits = ['1' if random() < 0.5 else '0' for _ in range(num_bits)]
  return ''.join(bits)


def euclidean_distance(v1, v2):
  return np.linalg.norm(v1 - v2, 2)

In [0]:
# Variation Operations

def point_mutation(bitstring, rate=None):
  
  if rate is None:
    rate = 1.0 / len(bitstring)
    
  child = ''
  
  for i in range(len(bitstring)):
    bit = bitstring[i]
    alternate_bit = '1' if bit == '1' else '0'
    child += alternate_bit if random() < rate else bit
    
  return child


def binary_tournament(pop):
  
  pop_len = len(pop)
  i, j = randint(0, pop_len - 1), randint(0, pop_len - 1)
  while i == j:  # Used to guarantee i and j are different.
    j = randint(0, pop_len - 1)
    
  return pop[i] if pop[i].fitness() < pop[j].fitness() else pop[j]


def crossover(parent1, parent2, rate):
  
  if random() >= rate:
    return '' + parent1
  
  child = ''
  for i in range(len(parent1)):
    
    child += parent1[i] if random() < 0.5 else parent2[i]
    
  return child

In [0]:
# Reproduction

def reproduce(selected, pop_size, p_cross):
  
  children = np.array([])
  for index, ind1 in enumerate(selected):
    ind2 = selected[index + 1] if index % 2 == 0 else selected[index - 1]
    if index == ( len(selected) - 1 ):
      ind2 = selected[0]
      
    child = Individual()
    child.set_bitstring( crossover(ind1.bitstring(), ind2.bitstring(), p_cross) )
    child.set_bitstring( point_mutation(child.bitstring()) )
    
    children = np.append(children, child)
    
    if len(children) > pop_size:
      break
      
  return children

In [0]:
# Individuals Evaluation

def calculate_objectives(pop, search_space, bits_per_param):
  
  for ind in pop:
    
    ind.set_vector( decode(ind.bitstring(), search_space, bits_per_param) )
    objectives = [
        f1(ind.vector())
      , f2(ind.vector())
    ]
    ind.set_objectives(objectives)
    

def is_dominant(ind1, ind2):
  
  ind1_objectives = ind1.objectives()
  ind2_objectives = ind2.objectives()
  
  for i in range(len(ind1_objectives)):
    if ind1_objectives[i] > ind2_objectives[i]:
      return False
    
  return True


def weighted_sum(ind):
  
  ind_objectives = ind.objectives()
  return np.sum(ind_objectives)


def calculate_dominated(pop):
  
  for ind in pop:
    if ind.dominated_set() is None:
      ind.set_dominated_set([])
    dominated_set = [
        dominated_ind for dominated_ind in ind.dominated_set()
                      if is_dominant(ind, dominated_ind) and ind != dominated_ind
    ]
    ind.set_dominated_set(dominated_set)
    

def calculate_raw_fitness(individual, pop):
  
  raw_individuals = [
      len(ind.dominated_set()) for ind in pop if is_dominant(ind, individual)
  ]    
      
  return sum(raw_individuals)


def calculate_density(individual, pop):
  
  for ind in pop:
    ind.set_distance(
        euclidean_distance(individual.objectives(), ind.objectives())
    )
  
  # Sorted by increasing distance.
  sorted_dist = np.sort(
    [ind.distance() for ind in pop]
  )
  
  k = int(np.sqrt(len(pop)))
  distance_sought = 1.0 / (sorted_dist[k] + 2.0)
  
  return distance_sought


def calculate_fitness(pop, archive, search_space, bits_per_param):
  
  calculate_objectives(pop, search_space, bits_per_param)
  
  union = np.append(pop, archive)
  
  calculate_dominated(union)
  
  for ind in union:
    ind.set_raw_fitness( calculate_raw_fitness(ind, union) )
    ind.set_density( calculate_density(ind, union) )

In [0]:
# Environmental Selection

def environmental_selection(pop, archive, archive_size):
  
  union = np.append(pop, archive)
  
  environment = [
      ind for ind in union if ind.fitness() < 1.0
  ]
  
  if len(environment) < archive_size:   # Fill environment.

    union = sorted(union, key = lambda ind: ind.fitness())
    for ind in union:
      if ind.fitness() >= 1.0:
        environment.append(ind)
      if len(environment) >= archive_size:
        break
        
  elif len(environment) > archive_size:  # Reduce environment.
    
    while len(environment) > archive_size:
      k = int(np.sqrt(len(environment)))
      for ind1 in environment:
        for ind2 in environment:
          ind2.set_distance(
              euclidean_distance(ind1.objectives(), ind2.objectives())
          )
        sorted_dist = np.sort(
           [ind.distance() for ind in pop]
        )
        ind1.set_density( sorted_dist[k] )
      environment = sorted(environment, key = lambda ind: ind.density())
      if len(environment) > 0:
        environment.pop(0)
        
  return environment

In [0]:
# Search Solution

def search(
    search_space, max_gens, pop_size, archive_size, p_cross, bits_per_param=16
):
  
  # Randomly generate initial population.
  pop = []
  for _ in range(pop_size):
    ind = Individual()
    ind.set_bitstring( random_bitstring(len(search_space) * bits_per_param) )
    pop.append(ind)
    
  pop = np.array(pop)
    
  generation, archive = 0, []
  
  while True:
        
    # Fitness measurement.
    calculate_fitness(pop, archive, search_space, bits_per_param)
    
    # Parent selection.
    archive = environmental_selection(pop, archive, archive_size)
    best = sorted(archive, key = lambda ind: weighted_sum(ind))
    best = best[0]
    
    print('Generation', generation, ' :: Best candidate found:', best.objectives(), ', f(x) =', sum(best.objectives()))
    
    if generation >= max_gens:
      break
      
    selected = []
    for _ in range(pop_size):
      selected.append(binary_tournament(archive))
      
    # Reproduction (Recombination / Mutation).
    pop = reproduce(selected, pop_size, p_cross)
    
    generation += 1
    
  return archive, pop, best

In [10]:
# Problem Configuration
problem_size = 1
search_space = [[-10, 10]]

# Algorithm Configuration
max_gens = 50
pop_size = 80
archive_size = 40  # Pareto set
p_cross = 0.90

# EXECUTE!
pareto_set, pop, sol = \
  search(search_space, max_gens, pop_size, archive_size, p_cross)

Generation 0  :: Best candidate found: [7.7031976  0.60134363] , f(x) = 8.304541228460934
Generation 1  :: Best candidate found: [1.1185237  0.88811247] , f(x) = 2.0066361669006367
Generation 2  :: Best candidate found: [2.04729877 0.32394483] , f(x) = 2.3712435973038763
Generation 3  :: Best candidate found: [2.38129613 0.2087166 ] , f(x) = 2.5900127309278247
Generation 4  :: Best candidate found: [2.27697439 0.24111569] , f(x) = 2.5180900815373928
Generation 5  :: Best candidate found: [2.45915615 0.18647742] , f(x) = 2.645633571277796
Generation 6  :: Best candidate found: [2.46011339 0.18621395] , f(x) = 2.6463273346354432
Generation 7  :: Best candidate found: [2.46011339 0.18621395] , f(x) = 2.6463273346354432
Generation 8  :: Best candidate found: [2.46777804 0.18411282] , f(x) = 2.6518908529509755
Generation 9  :: Best candidate found: [3.02012264 0.06872262] , f(x) = 3.0888452562588666
Generation 10  :: Best candidate found: [3.02012264 0.06872262] , f(x) = 3.0888452562588666


In [12]:
print(' --- Pareto Set --- ')
for i in range(len(pareto_set)):
  print(pareto_set[i].objectives())
  
print(' --- Population --- ')
for i in range(len(pop)):
  print(pop[i].objectives())
  
print('Solution:', sol.objectives(), ', f(x) =', sum(sol.objectives()))

 --- Pareto Set --- 
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[100. 144.]
[100. 144.]
[100. 144.]
[100. 144.]
[100. 144.]
[100. 144.]
[100. 144.]
[100. 144.]
[100. 144.]
[100. 144.]
[100. 144.]
[100. 144.]
[100. 144.]
[100. 144.]
[100. 144.]
[100. 144.]
[100. 144.]
[100. 144.]
[2.5053139  0.17404054]
 --- Population --- 
[77.19902197 46.05383238]
[85.65603474 52.63581654]
[100. 144.]
[100. 144.]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63581654]
[85.65603474 52.63

## Referencias

- (Brownlee, 2015) Brownlee, J. (2015). Strength Pareto Evolutionary Algorithm. [online]
Clever Algorithms. Available at: http://www.cleveralgorithms.com/nature-inspired/evolution/spea.html [Accessed 23 May 2019].
- (Zitzler & Thiele, 1998) Zitzler, E., & Thiele, L. (1998). An evolutionary algorithm for
multiobjective optimization: The strength pareto approach. TIK-report, 43.
- (Zitler et al., 2001) Zitzler, E., Laumanns, M., & Thiele, L. (2001). SPEA2: Improving the strength Pareto evolutionary algorithm. TIK-report, 103.
- (Deb et al., 2018)   K. Deb and A. Pratap and S. Agarwal and T. Meyarivan, "A Fast and Elitist Multiobjective Genetic Algorithm: NSGA–II", IEEE Transactions on Evolutionary Computation, 2002. 